**FULL COMPILED CODE** - PREDICTING SPOTIFY USER CHURN USING LISTENING BEHAVIOR DATA


In [ ]:
# LIBRARIES
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
import gradio as gr

# READING CSV FILE TAKEN FROM KAGGLE
df = pd.read_csv("/content/spotify_churn_dataset.csv")
print("Data loaded:", df.shape)
print(df.head())

# DROP USER ID (not needed for prediction)
df = df.drop(['user_id'], axis=1)

# ONE-HOT ENCODE CATEGORICAL COLUMNS
df = pd.get_dummies(df, columns=['subscription_type', 'country', 'top_genre'], drop_first=True)

# SPLIT FEATURES (X) & LABEL (y)
X = df.drop(['churned'], axis=1)
y = df['churned']

# SCALE NUMERIC DATA
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# SPLIT DATA FOR TRAINING
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# TRAIN MODEL
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# SAVE MODEL & SCALER
joblib.dump(model, 'rf_model.joblib')
joblib.dump(scaler, 'scaler.joblib')
joblib.dump(list(X.columns), 'feature_names.joblib')

# PREDICTION FUNCTION
def predict(subscription_type, country, avg_daily_minutes, number_of_playlists, top_genre,
            skips_per_day, support_tickets, days_since_last_login):

    # create small dataframe with 1 row
    new_row = pd.DataFrame([[subscription_type, country, avg_daily_minutes, number_of_playlists,
                             top_genre, skips_per_day, support_tickets, days_since_last_login]],
                           columns=['subscription_type','country','avg daily minutes','number_of_playlists',
                                    'top_genre','skips_per_day','support_tickets','days_since_last_login'])

    # apply one-hot encoding same way as training
    new_row = pd.get_dummies(new_row, columns=['subscription_type', 'country', 'top_genre'], drop_first=True)

    # match columns (if new input missing some dummy columns)
    feature_names = joblib.load('feature_names.joblib')
    for col in feature_names:
        if col not in new_row.columns:
            new_row[col] = 0
    new_row = new_row[feature_names]

    # scale and predict
    scaler_loaded = joblib.load('scaler.joblib')
    model_loaded = joblib.load('rf_model.joblib')
    new_scaled = scaler_loaded.transform(new_row)
    prob = model_loaded.predict_proba(new_scaled)[0][1]

    if prob > 0.5:
        return f"Might leave Spotify (Chance: {prob:.2%})"
    else:
        return f"Will probably stay (Chance: {prob:.2%})"

# GRADIO (PYTHON LIBRARY FOR CREATING WEB BASED INTERFACES)
with gr.Blocks() as demo:
    gr.Markdown("Spotify User Churn Prediction")

    sub_type = gr.Textbox(label="Subscription Type (Free/Premium/Family)", value="Free")
    country = gr.Textbox(label="Country", value="India")
    avg_minutes = gr.Number(label="Avg Daily Minutes", value=45)
    playlists = gr.Number(label="Number of Playlists", value=5)
    genre = gr.Textbox(label="Top Genre", value="Pop")
    skips = gr.Number(label="Skips per Day", value=2)
    tickets = gr.Number(label="Support Tickets", value=0)
    last_login = gr.Number(label="Days Since Last Login", value=5)

    btn = gr.Button("Predict")
    output = gr.Textbox(label="Result")

    btn.click(fn=predict,
              inputs=[sub_type, country, avg_minutes, playlists, genre, skips, tickets, last_login],
              outputs=output)

demo.launch(share=True)